In [82]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
from dash.dash_table.Format import Format
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "ABC321"
#database = "AAC"
#collection = "animals"
#host = "localhost"
#Port = 27017

# Connect to database via CRUD Module
db = AnimalShelter(username, password)
#shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # Top bar container
    html.Div(
        [
            # Logo positioned on left 
            html.Div(
                html.A(
                    href="https://www.snhu.edu/", target="_blank", 
                    children=[
                        html.Img(
                            alt="Logo made of line art of a dog's face",
                            src='data:image/png;base64,{}'.format(encoded_image.decode()),
                            style={'width':'20%','height':'auto', 'margin':'0'})
                    ]),
                style={'flex':'1', 'display':'flex', 'align-items':'center'}
            ),
            # Div for title of dashboard, centered
            html.Div(
                html.B(html.H1('CS-340 Dashboard')),
                style={'flex':'2', 'text-align':'center', 'display':'flex', 'align-items':'center', 'justify-content':'center'}),
            # Empty div for remaining layout
            html.Div(
                style={'flex':'1'}
            )
        ], 
        style={'display':'flex', 'width':'100%', 'align-items':'center'}
    ),
    
    html.Hr(),
    html.H4("Filter animals", style={'margin': '0px 0px 2px 10px'}),
    html.Div(className='buttonRow',
             style={'display' : 'flex', 'padding': '3px'},
             children=[
                 dcc.RadioItems(['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'Reset'], id='filter_type')]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                        data=df.to_dict('records'),
                        style_cell={'textAlign': 'left', 'padding': '5px', 'textOverflow': 'ellipsis', 'overflow': 'hidden'},
                        css=[{'selector': 'table', 'rule': 'table-layout: fixed'}],
                        # Style for header to make font bold and highlight a color difference with the rest of the table
                        style_header={'fontWeight': 'bold', 'backgroundColor': 'rgb(80,80,80)', 'color': 'white'},
                        editable = False,
                        filter_action = "native",
                        sort_action = "native",
                        sort_mode = "multi",
                        column_selectable = False,
                        row_selectable = 'single',
                        row_deletable = False,
                        selected_columns = [],
                        selected_rows = [0],
                        page_action = "native",
                        page_current = 0,
                        page_size = 20),

    
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6', style={'width':'49%','display':'inline-block'},
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',  style={'width':'49%','display':'inline-block'},
                 )
             ]),
    # Unique identifier
    html.Div(
        [html.H3("Ryan Hoskins, SNHU CS-340 MongDB Authentication")],
        style={'textAlign': 'center'})     
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter_type', 'value')])
def update_dashboard(filter_type):
    # Assign complete database as standard
    df = pd.DataFrame.from_records(db.read({})) 
    # If "Water Rescue" button is clicked, use read method to search for matching entries
    if (filter_type == "Water Rescue"):
        df = pd.DataFrame.from_records(
            db.read({"animal_type" : "Dog","breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
                     "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})
        )
    # If "Mountain Rescue" button is clicked, use read method to search for matching entries
    elif (filter_type == "Mountain Rescue"):
        df = pd.DataFrame.from_records(
            db.read({"animal_type" : "Dog", "breed":{"$in":["German Shepard","Alaskan Malamute","Old English Sheepdog", "Siberian Husky","Rottweiler"]},
                     "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks": {"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})
        ) 
    # If "Disaster Rescue" button is clicked, use read method to search for matching entries
    elif (filter_type == "Disaster Rescue"):
        df = pd.DataFrame.from_records(
            db.read({"animal_type" : "Dog","breed":{"$in":["Doberman Pinscher","German Shepard","Golden Retriever","Bloodhound","Rottweiler"]},
                     "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}})
        )
    # Default option to enable user to backtrack to complete database if needed after using it
    elif (filter_type == "Reset"):
        df = pd.DataFrame.from_records(db.read({})) 
        
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')
# FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
# Update graph using viewData
def update_graphs(viewData):
    if viewData is None:
        return;
    df2 = pd.DataFrame.from_dict(viewData)
    # Set captions and type of graph
    figure = px.pie(df2, names='breed', title='Animal Breeds')
    # Prevent overflow 
    figure.update_traces(textposition='inside')
    figure.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    return dcc.Graph(figure = figure)
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "data"),
     Input('datatable-id', "selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Successfully connected
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
Dash app running on http://127.0.0.1:8050/
